# NCR District 4 Cell Tower Decision Support

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

import matplotlib.pyplot as plt
import seaborn as sns

# Part 1: Problem Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in C$: Index and set of cities.

$G(T,C,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower,  the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in C$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at city $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if city $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function(s)

- **Population covered**. We seek to maximize the total population covered by the towers.

\begin{equation}
\max\sum_{j \in C} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### Constraints

- **Coverage**. For each city $j \in R$ ensure that at least one tower that covers a city must be selected.

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Las Piñas | Makati | Muntinlupa | Parañaque | Pasay | Pateros | Taguig |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Tower 0 | - | 1 | 1 | - | - | 1 | - |
| Tower 1 | 1 | 1 | - | 1 | 1 | - | - |
| Tower 2 | 1 | 1 | 1 | 1 | - | - | 1 |
| Tower 3 | 1 | 1 | 1 | - | 1 | - | 1 |


# Part 2: Model Encoding

### Retrieve Parameter Data

In [48]:
## Budget
budget = 34.7 ## million pesos

In [49]:
## Population Estimates
estimates = pd.read_csv('data/dist4_2025_estimates.csv')
population = estimates.population.values
## cities
cities = np.arange(len(estimates.city))

In [50]:
## Tower Costs (in Million PHP)
tower_cost = pd.read_csv('data/tower_cost.csv')
cost = tower_cost.cost.values
tower_cost

,tower,cost
0,0,8.81
1,1,7.77
2,2,8.36
3,3,12.70


In [51]:
## Tower Coverage
sites, coverage = gp.multidict({
    0: [[1, 2, 5]],
    1: [[0, 1, 3, 4]],
    2: [[0, 1, 2, 3, 6]],
    3: [[0, 1, 2, 4, 6]]
})

## Model Setup

In [52]:
m = gp.Model("cell_tower")

## Variable Declaration

In [53]:
build = m.addMVar(len(sites), vtype=GRB.BINARY, name="build")
covered = m.addMVar(len(cities), vtype=GRB.BINARY, name="covered")

## Add Constraints

In [54]:
m.addConstrs((gp.quicksum(build[t] for t in sites if r in coverage[t]) >= covered[r]
                        for r in cities), name="build2cover")
m.addConstr(build@cost <= budget, name="budget")

<MConstr () *awaiting model update*>

## Add Objective Function

In [55]:
m.setObjective(covered@population, GRB.MAXIMIZE)

## Optimize the MIP

In [56]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 5 5600H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 11 columns and 28 nonzeros
Model fingerprint: 0xf8b7c7b0
Variable types: 0 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [7e+04, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 5 rows, 8 columns, 17 nonzeros
Found heuristic solution: objective 4194938.0000
Variable types: 0 continuous, 8 integer (8 binary)

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 2: 4.19494e+06 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.19493800000

# Part 3: Analysis

The result of the optimization model shows that the maximum number of subscribers can be covered with PHP 34.7M is 4,194,938 people

We can determine the suggested site plan by retrieving the values from the model.

In [60]:
print('Recommended towers to build:')
for i in range(len(build.X)):
    if build.X[i] == 1:
        print(f'Tower {i}')

Recommended towers to build:
Tower 0
Tower 1
Tower 2


### Tower Coverage %

In [62]:
total_population = population.sum()

coverage_pct = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage of the current cell tower build plan is: {coverage_pct} %")


 The population coverage of the current cell tower build plan is: 100.0 %


### Budget Utilization

In [66]:
total_cost = build.X@cost
budget_util = round(100*total_cost/budget, 2)

print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_util} %")


 The percentage of budget consumed associated to the cell towers build plan is: 71.87 %
